In [1]:
#coding:utf-8
%reload_ext autoreload 
%autoreload 2
import os
import sys

project_basedir = '..'
sys.path.append(project_basedir)
from cchess_zero import cbf
from cchess_zero import board_visualizer
from matplotlib import pyplot as plt
import os
from cchess_zero.gameboard import *
from net import resnet
import common
from gameplays import gameplay,players
from gameplays.analysis import get_km_json
import json
%matplotlib inline

/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

fullpath = '../data/distributed/2018-07-24_23-54-28_750_mcts-mcts_net-net_w.cbf'
moves = cbf.cbf2move(fullpath)
fname = fullpath.split('/')[-1]
print(fname)
state = gameplay.GameState()

2018-07-24_23-54-28_750_mcts-mcts_net-net_w.cbf


In [14]:
from PIL import Image
import imageio
imgs_for_gif = []
state = gameplay.GameState()
statestr = 'RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c1/9/rnbakabnr'
for i in moves[:-1]: 
    statestr = GameBoard.sim_do_action(i,statestr)
    imgs_for_gif.append(board_visualizer.get_board_img(statestr))
    state.do_move(i)
imgs_for_gif = imgs_for_gif[-10:]
imageio.mimsave('../data/gif/{}-.gif'.format(fname[:-4]), imgs_for_gif,duration=0.5)

In [15]:
i,len(moves)

('g4g5', 38)

In [16]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [17]:
display_gif('../data/gif/{}-.gif'.format(fname[:-4]))

# ANALYSIS

In [19]:
network = resnet.get_model(os.path.join(project_basedir,
    '../data/prepare_weight/2018-07-21_21-13-25'[3:]),
    common.board.create_uci_labels(),GPU_CORE=[None],FILTERS=128,NUM_RES_LAYERS=7)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from ../data/prepare_weight/2018-07-21_21-13-25


In [20]:
network_player = players.NetworkPlayer(state.currentplayer,network,n_playout=400)

In [21]:
network_player.make_move(state=state,actual_move=False)

('d7e7', -0.596575206083944)

In [26]:
dd.virtual_loss

0.0

In [33]:
childs = network_player.mcts_policy._root._children
for move in childs:
    val = childs[move]
    print(move,val._n_visits,val._P)

a3b3 5 0.00027760916
a3a2 26 2.5102796e-05
b9a7 5 0.00018872795
b9c7 5 0.0004668583
b6a6 4 5.6672598e-06
b6c6 6 6.117904e-05
b6d6 6 9.843273e-05
b6e6 1 0.01561201
b6f6 6 0.00019385909
b6b8 4 9.4502444e-05
b6b7 5 0.00015958751
b6b5 6 0.0001375926
b6b4 6 0.000100078876
b6b3 6 4.4244276e-05
b6b2 3 0.0004103136
b6b1 3 4.7956222e-05
c8a8 5 6.500514e-06
c8b8 5 0.00025674148
c8d8 5 0.00071586954
c8e8 2 0.009063241
c8f8 5 0.0036557636
c8c7 5 0.00019559282
c8c6 7 0.00018979666
c5c4 6 0.0019809213
d7a7 6 4.8103622e-05
d7b7 5 5.999132e-05
d7c7 5 0.00017222496
d7e7 78 0.6773214
d7f7 5 0.014502326
d7d8 5 0.0002342753
d7d6 4 0.00018197799
d7d5 7 0.00012444807
d7d4 4 0.0003885164
d7d3 5 0.0008969135
d7d2 3 0.00088837644
d7d1 7 9.842475e-05
d7d0 7 5.165443e-05
e9e8 17 0.011022366
g8d8 5 5.981217e-05
g8e8 1 0.00082954066
g8f8 4 9.393528e-05
g8h8 3 7.248808e-05
g8i8 4 7.305863e-05
g7e8 1 0.00029504974
g7e6 2 0.0020503758
g7i8 4 3.7349375e-05
g7i6 3 1.0771588e-05
g6g5 54 0.14083758
i5i4 9 0.048541863
d9e

In [28]:
datajson = get_km_json(network_player.mcts_policy,state.statestr,depth=40)

In [29]:
with open('../data/mstc_vis/sample.km','w') as whdl:
    json.dump(datajson,whdl)